In [1]:
#Nettoyage de données pour les validations par station et horaires: pour chaque station, on a le pourcentage d'affluence à chaque horaire sur une journée type
import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = 'http://opendata.stif.info/explore/dataset/validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false'
données_validation= pd.read_csv(url)
données_validation = pd.read_csv(url, delimiter=';')
données_validation_station_horaire = données_validation[['libelle_arret', 'trnc_horr_60','pourc_validations']]
donnees_organisees = données_validation_station_horaire.pivot_table(index='libelle_arret', columns='trnc_horr_60', values='pourc_validations', aggfunc='mean').reset_index()
print(donnees_organisees)

trnc_horr_60    libelle_arret   0H-1H  10H-11H  11H-12H  12H-13H  13H-14H  \
0                    ABBESSES  1.9100    3.792    4.776    5.672    6.022   
1                       ABLON  0.0375    5.314    5.110    5.018    4.816   
2               ACHERES GRAND  0.1200    4.506    5.692    5.470    6.676   
3               ACHERES VILLE  0.2100    5.934    6.148    6.384    6.170   
4               AEROPORT CDG1  0.1260    4.102    4.486    5.208    6.560   
..                        ...     ...      ...      ...      ...      ...   
742                    VOSVES     NaN    4.360    4.218    9.368    4.512   
743           VULAINES SUR SE     NaN    7.312    4.808    5.806    6.052   
744                    WAGRAM  0.7640    5.088    5.930    6.988    6.962   
745                    YERRES  0.1200    6.058    6.192    5.880    5.788   
746            YVRIS NOISY GD  0.0000    4.936    5.674    5.322    5.284   

trnc_horr_60  14H-15H  15H-16H  16H-17H  17H-18H  ...    23H-0H     2H-3H  

In [2]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = 'http://opendata.stif.info/explore/dataset/validations-reseau-ferre-nombre-validations-par-jour-1er-semestre/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')
print(données_validation2)

#On extrait les données que pour le mois de mai, à défaut d'avoir 
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'])
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()
print(donnees_sum)

               jour  code_stif_trns code_stif_res code_stif_arret  \
0        2023-06-26             800           853             568   
1        2023-06-26             800           853             568   
2        2023-06-26             800           853             568   
3        2023-06-26             800           853             568   
4        2023-06-26             800           853             595   
...             ...             ...           ...             ...   
1096204  2023-03-01             100           110             842   
1096205  2023-03-01             100           110             848   
1096206  2023-03-01             100           110             848   
1096207  2023-03-01             100           110             849   
1096208  2023-03-01             100           110             850   

         libelle_arret    lda categorie_titre  nb_vald  
0            MONTSOULT  67000               ?       28  
1            MONTSOULT  67000       AMETHYSTE        9  


KeyError: 'JOUR'